In [1]:
import requests as r 

resp = r.get("https://raw.githubusercontent.com/walkerdb/supreme_court_transcripts/master/oyez/case_summaries.json")

In [3]:
len(resp.json())

8137

In [4]:
validCases = []

for case in resp.json():
    caseDetails = r.get(case["href"]).json()
    if type(caseDetails) == dict:
        if (caseDetails["facts_of_the_case"] is not None and caseDetails["facts_of_the_case"] != '') and (caseDetails["question"] is not None and caseDetails["question"] != '') and (caseDetails["decisions"] is not None and caseDetails["decisions"] != ''):
            validCases.append(caseDetails)
    

In [5]:
len(validCases)

187

In [10]:
def parseCase(case):
    try:
        cn = case["name"].split("v.")
        plantiff = cn[0].strip()
        defendant = cn[1].strip()
        question = case["question"].strip().strip("<p>").strip("</p>").strip("\n")
        facts = case["facts_of_the_case"].strip().strip("<p>").strip("</p>").strip("\n")
        winningParty = case["decisions"][0]["winning_party"]
        inFavorPlantiff = True if winningParty == plantiff or winningParty == case["first_party"] else False
        votes = {}
        for vote in case["decisions"][0]["votes"]:
            if vote["vote"] != "none":
                if inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "plantiff"
                elif not inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif not inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "plantiff"
        return {
            "plantiff": plantiff,
            "defendant": defendant,
            "question": question,
            "facts": facts,
            "winningParty": winningParty,
            "inFavorPlantiff": inFavorPlantiff,
            "votes": votes
        }
    except:
        pass

In [15]:
processedCases = []
for case in validCases:
    paresed = parseCase(case)
    if paresed is not None:
        processedCases.append(paresed)

In [16]:
len(processedCases)

186

In [17]:
processedCases[0]

{'plantiff': 'Roe',
 'defendant': 'Wade',
 'question': "Does the Constitution recognize a woman's right to terminate her pregnancy by abortion?",
 'facts': 'In 1970, Jane Roe (a fictional name used in court documents to protect the plaintiff’s identity) filed a lawsuit against Henry Wade, the district attorney of Dallas County, Texas, where she resided, challenging a Texas law making abortion illegal except by a doctor’s orders to save a woman’s life. In her lawsuit, Roe alleged that the state laws were unconstitutionally vague and abridged her right of personal privacy, protected by the First, Fourth, Fifth, Ninth, and Fourteenth Amendments.',
 'winningParty': 'Jane Roe',
 'inFavorPlantiff': True,
 'votes': {'william_o_douglas': 'plantiff',
  'potter_stewart': 'plantiff',
  'thurgood_marshall': 'plantiff',
  'william_j_brennan_jr': 'plantiff',
  'byron_r_white': 'defendant',
  'warren_e_burger': 'plantiff',
  'harry_a_blackmun': 'plantiff',
  'lewis_f_powell_jr': 'plantiff',
  'willia

In [19]:
import pickle

with open('processedCases.pickle', 'wb') as handle:
    pickle.dump(processedCases, handle, protocol=pickle.HIGHEST_PROTOCOL)